In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
import requests
import json
API_TOKEN  = r"GRjdYvo48gh7lYepaYgUTVtknrzqdXeuBWHyQPdzlVoMC6e3874VHwQQzw6k"

# PUT requests to change the sales status of existiong jobs
def update_sales_status(API_TOKEN, uuid, name, status):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        "sales_status": status
    }
    response = requests.put(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")

# PUT requests to change the contract date
def update_contract_date(API_TOKEN, uuid, name, new_date):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        'contract_date': new_date
    }
    response = requests.put(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")
#creat df from data stored in followup
def get_df(API_TOKEN, page):
    # url = 'https://app.followupcrm.com/api/v1/file-data'
    url = f'https://app.followupcrm.com/api/v1/file-data?page={page}'
    params = {
        'api_token': API_TOKEN,
        'per_page': 15
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
        # print(len(df))
        return df, json_data
    else:
        print(f"Error {response.status_code}: {response.text}")
# creat and return a subset based on age of the bid and list of sales statuses 
def reduce_dataset(full_df, d_count, status_list):
    full_df['bid_date'] = pd.to_datetime(full_df['bid_date'])
    reduced = full_df[full_df['sales_status'].isin(status_list)]
    reduced = reduced.assign(time_passed = lambda x: pd.Timestamp(date.today()) - x['bid_date'])
    reduced['notify'] = reduced['time_passed'].apply(lambda y: 1 if y > pd.Timedelta(f'{d_count} days 00:00:00') else 0)
    notify = reduced[reduced['notify'] == 1]
    return notify
# subset if specified column is NA 
def missing_value(full_df, field):
    reduced = full_df[full_df[field].isna()]
    return reduced

# formated strings for emails

def fill_string1(estimator, project_name, current_status, days_past, contact, cell, email):
    status_past_due = f"""
    What's up, {estimator},
    
    It appears {project_name} has had the status of {current_status} for {days_past} past the bid date.
    
    Please consider updating this status or call {contact}'s cell({cell})' or email {email}.
    
    This message will repeat weekly until changes are made.
    
    Thank you!
    Fiore-up"""
    return status_past_due
def fill_strin2(estimator, project_name):
    no_bid_date = f"""
    What's up, {estimator},
    
    It appears {project_name} doesn't have a bid date, please update this in follow-up.
    
    This message will repeat weekly until changes are made.
    
    Thank you!
    Fiore-up"""
    return no_bid_date

def status_notify(df):
    for ind in range(0,len(df)):
        current = df.iloc[ind]
        estimator, project_name, current_status, days_past, contact, cell, email = current['estimator'],current['name'], current['sales_status'], current['time_passed'], current['contact.name'],current['contact.cell'],current['contact.email']
        f_string = fill_string1(estimator, project_name, current_status, days_past, contact, cell, email)
        # email send aspect
    print(f_string)

def bid_date_notify(df):
    for ind in range(0,len(df)):
        current = df.iloc[ind]
        estimator, project_name, current_status, days_past, contact, cell, email = current['estimator'],current['name'], current['sales_status'], current['time_passed'], current['contact.name'],current['contact.cell'],current['contact.email']
        f_string = fill_string2(estimator, project_name)
        # email send aspect
    print(f_string)


fiore_contacts = pd.DataFrame.from_dict({'estimator':['Al Barker', 'Angie Cole', 'Conner Spinar', 'Garrett  Hammers', 'Jaime Astorga', 'Joel Holland',"Neil O'Grady", 'Rob Slauson', 'Ryan Frank', 'Sara Locke', 'Stephen Ceresa', 'Trevor Lepke', 'Will Davis', 'Will Mcclure'
], 'est_email': [ 'abarker@fioreandsons.com', 'angie.cole@fioreandsons.com', 'conner.spinar@fioreandsons.com', 'ghammers@fioreandsons.com', 'jaime.astorga@fioreandsons.com','jholland@fioreandsons.com', 'nogrady@fioreandsons.com', 'rslauson@fioreandsons.com', 'rfrank@fioreandsons.com', 'sara.locke@fioreandsons.com', 'stephen.ceresa@fioreandsons.com', 'trevor.lepke@fioreandsons.com', 'will.davis@fioreandsons.com', 'wmcclure@fioreandsons.com',
]})

In [3]:
%%time
# creates a dataframe of all jobs in followup(pulled from API)
df_list = []
json = get_df(API_TOKEN, 1)[1]
last_page = json['meta']['last_page']
for i in range(1,last_page+1):
    df_list.append(get_df(API_TOKEN, i)[0])
# combine dfs
full_df = pd.concat(df_list)
reference = full_df
#merge with df of estimators emails
full_df = full_df.merge(fiore_contacts, on='estimator')
# subsets to trigger notifications
submits = reduce_dataset(full_df, '14', ["Bidding","New Lead","Submitted"])
budgets = reduce_dataset(full_df, '90', ["Budget"])
no_bid_date = missing_value(full_df, 'bid_date')

CPU times: total: 312 ms
Wall time: 54.2 s


In [ ]:
full_df = full_df[['name', 'estimator', 'sales_status', 'bid_date', ]]

In [13]:
# test f strings
test_df = submits[submits['estimator']== 'Conner Spinar']
test_df = test_df[['name']]
test_f = f""" 
testing out the spacing
below:
{test_df}

hhsdflakshfasdhihgf
sajfsdjsadj
"""
test_f

' \ntesting out the spacing\nbelow:\n                                                  name\n76                    1565 Colorado 155 Unit Apartment\n99                                 Sixt Rent-A-Car DIA\n115  Westminster Municipal Court & Community Reach ...\n145                 Gatherings at Parker - Life Church\n165                               Walmart Fuel Station\n186               US 85 Resurfacing: I-76 To 124th Ave\n187  Brighton High School Student Parking Lot Repla...\n190                            Brakes Plus Westminster\n193                          CCD 51st Ave Improvements\n343               Adams County 2025 Street Resurfacing\n450  Westminster Municipal Court & Community Reach ...\n451                           Uplands Filing 1 Block 3\n\nhhsdflakshfasdhihgf\nsajfsdjsadj\n'

In [29]:
test_df = submits[submits['estimator']== 'Conner Spinar']
test_df['str'] = test_df['name'] + ' , ' + test_df['sales_status'] + ' , ' + test_df['time_passed'].astype(str)
test_str = list(test_df['str'])
# new_str = (repr(test_str).replace(',', ',\n'))
formatted_string = "\n".join(test_str)

C:\Users\will.davis\AppData\Local\Temp\ipykernel_25028\1972325380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['str'] = test_df['name'] + ' , ' + test_df['sales_status'] + ' , ' + test_df['time_passed'].astype(str)


In [30]:
test_f = f""" 
testing out the spacing
below:
{formatted_string}

hhsdflakshfasdhihgf
sajfsdjsadj
"""
print(test_f)

 
testing out the spacing
below:
1565 Colorado 155 Unit Apartment , Submitted , 109 days
Sixt Rent-A-Car DIA , Submitted , 91 days
Westminster Municipal Court & Community Reach Center , Submitted , 69 days
Gatherings at Parker - Life Church , Submitted , 56 days
Walmart Fuel Station , Submitted , 56 days
US 85 Resurfacing: I-76 To 124th Ave , Submitted , 55 days
Brighton High School Student Parking Lot Replacement , Submitted , 53 days
Brakes Plus Westminster , Submitted , 48 days
CCD 51st Ave Improvements , Submitted , 38 days
Adams County 2025 Street Resurfacing , Submitted , 46 days
Westminster Municipal Court & Community Reach Center , Submitted , 18 days
Uplands Filing 1 Block 3 , Submitted , 18 days

hhsdflakshfasdhihgf
sajfsdjsadj



In [41]:
# change subset to status update to notify people
# for ind in range(0,len(submits)):
#     current = submits.iloc[ind]
#     estimator, project_name, current_status, days_past, contact, cell, email = current['estimator'],current['name'], current['sales_status'], current['time_passed'], current['contact.name'],current['contact.cell'],current['contact.email']
#     f_string = fill_string1(estimator, project_name, current_status, days_past, contact, cell, email)
# print(f_string)
status_notify(submits)


    Greetings, Ryan Frank,
    
    It appears Project Red Hawk has had the status of Submitted for 25 days 00:00:00 past the bid date.
    
    Please consider updating this status or call Trenton Reynolds's cell()' or email trenton.reynolds@catamountinc.com.
    
    This message will repeat weekly until changes are made.
    
    Thank you!
    Fiore-up


## Export to Excel (should run monthly)

In [19]:
# make path for each day that the data is compiled 
end_file = f'followup_data_{str(date.today())}.xlsx'
end_path = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P5 Followup API\follow_up_data\{}".format(end_file)
# create excel for record keeping
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
reference.to_excel(writer, sheet_name="full_df")
# pivot_table.to_excel(writer, sheet_name="Pivot")
writer.close()

# data set of old budgets to become budget loss

In [ ]:
# data set of old budgets to become budget loss
df2 = df
df2['bid_date'] = pd.to_datetime(df2['bid_date'], format='%m/%d/%Y')
in_range = df2[df2['bid_date'] < pd.to_datetime("2024-12-31")]
# I only want budgets
just_buds = in_range[in_range['sales_status'] == 'Budget']
len(just_buds)
# in_range = df[df['']]
for ind in range(0,len(just_buds)):
    current = just_buds.iloc[ind]
    # print(current['uuid'], current['name'])
    update_sales_status(API_TOKEN, current['uuid'], current['name'], 'Budget Loss')
    print(ind)

# data set of won jobs to update contract date 

In [ ]:
df3 = df
df3['bid_date'] = pd.to_datetime(df3['bid_date'], format='%m/%d/%Y')
df3['dead_date'] = pd.to_datetime(df3['dead_date']).dt.date
# df3['dead_date'] = df3['dead_date'].astype('datetime64[D]')
in_range = df3[df3['bid_date'] < pd.to_datetime("2024-10-15")]
# I only want budgets
just_wins = in_range[in_range['sales_status'] == 'Won']
need_contract_date = just_wins.dropna(subset=['dead_date'])
# need_contract_date = just_wins[just_wins['dead_date'] != pd.NaT]
print(len(just_wins), len(need_contract_date))
for ind in range(0,len(need_contract_date)):
    current = need_contract_date.iloc[ind]
    # print(current['uuid'], current['name'], str(current['dead_date']))
    update_contract_date(API_TOKEN, current['uuid'], current['name'], current['dead_date'])
    print(ind)


In [7]:
df.columns

Index(['id', 'uuid', 'team_id', 'name', 'address', 'city', 'state', 'zip',
       'sales_behaviours', 'account_manager', 'estimator', 'companies',
       'proposal_delivered', 'sales_status', 'office', 'Lead_From',
       'contract_amount', 'base_bid', 'contract_date', 'contract_type',
       'bid_date', 'last_update', 'created_at', 'external_id',
       'external_vendor', 'custom_fields', 'notes', 'outlook_conversation_id',
       'estimated_contract_date', 'gross_margin_percentage', 'labor_hours',
       'dead_date', 'marketing_status', 'pre_bid', 'take_off_person',
       'products_and_services', 'client_profile', 'relate_and_connect',
       'rules_of_engagement', 'find_problems', 'money_step',
       'decision_process', 'change_orders', 'record_detail_custom_fields',
       'contact.name', 'contact.phone', 'contact.fax', 'contact.cell',
       'contact.email'],
      dtype='object')